In [4]:
import numpy as np
import pandas as pd
import plotly.express as px
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
print(os.environ.get("DATA_SOURCE_DIR"))

data/


In [6]:
DATA_SOURCE = "../data/5_yr_data/"

In [7]:
up_5yrs = pd.read_csv(DATA_SOURCE+"UP5_years.csv")

In [8]:
up_5yrs.head()

,Unnamed: 0,district_name,market_name,commodity,variety,grade,min_rs_quintal,max_rs_quintal,modal_rs_quintal,date,year,month,day_of_month
0,446185,Auraiya,Achalda,Potato,Badshah,FAQ,600.0,700.0,670.0,04 Oct 2019,2019,Oct,4
1,446186,Auraiya,Achalda,Potato,Badshah,FAQ,780.0,820.0,800.0,05 Dec 2019,2019,Dec,5
2,446187,Auraiya,Achalda,Potato,Badshah,FAQ,850.0,920.0,900.0,17 Dec 2019,2019,Dec,17
3,446188,Auraiya,Achalda,Potato,Badshah,FAQ,890.0,950.0,900.0,18 Oct 2019,2019,Oct,18
4,446189,Auraiya,Achalda,Potato,Badshah,FAQ,1000.0,1090.0,1060.0,29 Jan 2020,2020,Jan,29


In [9]:
up_5yrs.drop(columns=['Unnamed: 0'], inplace=True)

In [10]:
up_5yrs.head()

,district_name,market_name,commodity,variety,grade,min_rs_quintal,max_rs_quintal,modal_rs_quintal,date,year,month,day_of_month
0,Auraiya,Achalda,Potato,Badshah,FAQ,600.0,700.0,670.0,04 Oct 2019,2019,Oct,4
1,Auraiya,Achalda,Potato,Badshah,FAQ,780.0,820.0,800.0,05 Dec 2019,2019,Dec,5
2,Auraiya,Achalda,Potato,Badshah,FAQ,850.0,920.0,900.0,17 Dec 2019,2019,Dec,17
3,Auraiya,Achalda,Potato,Badshah,FAQ,890.0,950.0,900.0,18 Oct 2019,2019,Oct,18
4,Auraiya,Achalda,Potato,Badshah,FAQ,1000.0,1090.0,1060.0,29 Jan 2020,2020,Jan,29


In [11]:
up_5yrs.shape

(1226694, 12)

## Missing Values

In [12]:
up_5yrs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226694 entries, 0 to 1226693
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   district_name     1226694 non-null  object 
 1   market_name       1226694 non-null  object 
 2   commodity         1226694 non-null  object 
 3   variety           1226694 non-null  object 
 4   grade             1226694 non-null  object 
 5   min_rs_quintal    1226694 non-null  float64
 6   max_rs_quintal    1226694 non-null  float64
 7   modal_rs_quintal  1226694 non-null  float64
 8   date              1226694 non-null  object 
 9   year              1226694 non-null  int64  
 10  month             1226694 non-null  object 
 11  day_of_month      1226694 non-null  int64  
dtypes: float64(3), int64(2), object(7)
memory usage: 112.3+ MB


### There are no missing values in any column
### Need to check where we are populating with zero when data is not present

In [13]:
up_5yrs.describe()

,min_rs_quintal,max_rs_quintal,modal_rs_quintal,year,day_of_month
count,1.226694e+06,1.226694e+06,1.226694e+06,1.226694e+06,1.226694e+06
mean,1.704229e+03,1.821545e+03,1.772429e+03,2.020855e+03,1.572314e+01
std,7.698786e+02,7.849455e+02,7.662350e+02,1.934080e+00,8.784076e+00
min,0.000000e+00,0.000000e+00,1.100000e+01,2.018000e+03,1.000000e+00
25%,1.075000e+03,1.200000e+03,1.150000e+03,2.019000e+03,8.000000e+00
50%,1.725000e+03,1.830000e+03,1.785000e+03,2.021000e+03,1.600000e+01
75%,2.280000e+03,2.400000e+03,2.340000e+03,2.023000e+03,2.300000e+01
max,1.050000e+04,2.650000e+04,1.054000e+04,2.024000e+03,3.100000e+01


### Outlier Detection

In [14]:
def get_price_distribution_for_commodity(field, df, commodity):
    field_values = df[df['commodity'] == commodity][field]
    
    price_distribution = {
        'commodity': commodity,
        'min_value': field_values.min(),
        '1st_percentile':field_values.quantile(0.01),
        '5th_percentile': field_values.quantile(0.05),
        '25th_percentile': field_values.quantile(0.25),
        '50th_percentile': field_values.quantile(0.50),
        '75th_percentile': field_values.quantile(0.75),
        '90th_percentile': field_values.quantile(0.90),
        '95th_percentile': field_values.quantile(0.95),
        '99th_percentile': field_values.quantile(0.99),
        '99.5th_percentile': field_values.quantile(0.995),
        'max_value': field_values.max()
    }
    return price_distribution

In [15]:
def get_price_distribution_for_all_commodities(field, df):
    price_distribution_for_all_commodities = []
    unique_commodities = set(df.commodity.values)

    for commodity in unique_commodities:
        price_distribution_for_commodity = get_price_distribution_for_commodity(field, df, commodity)
        price_distribution_for_all_commodities.append(price_distribution_for_commodity)

    df_price_distribution = pd.DataFrame(price_distribution_for_all_commodities)
    return df_price_distribution

In [16]:
get_price_distribution_for_all_commodities('min_rs_quintal', up_5yrs)

,commodity,min_value,1st_percentile,5th_percentile,25th_percentile,50th_percentile,75th_percentile,90th_percentile,95th_percentile,99th_percentile,99.5th_percentile,max_value
0,Wheat,0.0,1450.0,1570.0,1750.0,1900.0,2150.0,2370.0,2450.0,2650.0,2710.000,7020.0
1,Potato,0.0,300.0,450.0,700.0,945.0,1290.0,1700.0,2200.0,2900.0,3009.175,7400.0
2,Rice,0.0,1700.0,2050.0,2300.0,2440.0,2600.0,2850.0,2950.0,3300.0,4715.700,9800.0
3,Onion,0.0,520.0,750.0,1100.0,1450.0,2050.0,2900.0,3400.0,4442.6,5000.000,10500.0


#### Clearly, we could observe the following abnormal readings:
- ##### Min value being zero 
    - it is not normal for the price of 100kgs of any commodity to be zero 
    - may be due to missing values being imputed with zero
- ##### Max value is 2 to 3 times the value in 99.5 percentile
    - it is not normal for the max price to be 2 or 3 times the 99.5 percentile
    - may be due to wrong data entry or exceptional events 

In [17]:
get_price_distribution_for_all_commodities('max_rs_quintal', up_5yrs)

,commodity,min_value,1st_percentile,5th_percentile,25th_percentile,50th_percentile,75th_percentile,90th_percentile,95th_percentile,99th_percentile,99.5th_percentile,max_value
0,Wheat,0.0,1511.0,1640.0,1840.0,1975.0,2270.0,2460.0,2550.0,2750.0,2810.00,16700.0
1,Potato,0.0,400.0,550.0,800.0,1060.0,1400.0,1850.0,2360.0,3050.0,3200.00,11200.0
2,Rice,0.0,1820.0,2185.0,2400.0,2550.0,2725.0,3000.0,3100.0,3480.0,4797.85,26500.0
3,Onion,0.0,600.0,850.0,1250.0,1600.0,2200.0,3100.0,3600.0,4650.0,5250.00,11050.0


In [18]:
get_price_distribution_for_all_commodities('modal_rs_quintal', up_5yrs)

,commodity,min_value,1st_percentile,5th_percentile,25th_percentile,50th_percentile,75th_percentile,90th_percentile,95th_percentile,99th_percentile,99.5th_percentile,max_value
0,Wheat,17.0,1525.0,1620.0,1800.0,1935.0,2220.0,2415.0,2500.0,2700.0,2760.0,7040.0
1,Potato,11.0,400.0,510.0,760.0,1000.0,1350.0,1775.0,2300.0,2970.0,3100.0,9050.0
2,Rice,21.0,1860.0,2135.0,2355.0,2500.0,2665.0,2940.0,3005.0,3380.0,4750.0,9820.0
3,Onion,88.0,640.0,800.0,1190.0,1530.0,2140.0,3000.0,3500.0,4500.0,5100.0,10540.0


## Analyse Onion Prices

In [23]:
price_distribution = get_price_distribution_for_commodity("modal_rs_quintal", up_5yrs, "Onion")
df_price_distribution = pd.DataFrame([price_distribution])
df_price_distribution

,commodity,min_value,1st_percentile,5th_percentile,25th_percentile,50th_percentile,75th_percentile,90th_percentile,95th_percentile,99th_percentile,99.5th_percentile,max_value
0,Onion,88.0,640.0,800.0,1190.0,1530.0,2140.0,3000.0,3500.0,4500.0,5100.0,10540.0


In [26]:
up_5yrs[(up_5yrs['year'] == 2018)&(up_5yrs['month'] == "Apr")& (up_5yrs['commodity'] == 'Onion')&(up_5yrs['district_name'] == "Ayodhya")].sort_values(by="day_of_month")

,district_name,market_name,commodity,variety,grade,min_rs_quintal,max_rs_quintal,modal_rs_quintal,date,year,month,day_of_month
726959,Ayodhya,Faizabad,Onion,Red,FAQ,1150.0,1300.0,1250.0,01 Apr 2018,2018,Apr,1
735490,Ayodhya,Faizabad,Onion,Red,FAQ,1100.0,1300.0,1200.0,02 Apr 2018,2018,Apr,2
717485,Ayodhya,Faizabad,Onion,Red,FAQ,1000.0,1200.0,1125.0,04 Apr 2018,2018,Apr,4
726958,Ayodhya,Faizabad,Onion,Red,FAQ,1000.0,1200.0,1100.0,05 Apr 2018,2018,Apr,5
725993,Ayodhya,Faizabad,Onion,Red,FAQ,1000.0,1200.0,1100.0,06 Apr 2018,2018,Apr,6
734720,Ayodhya,Faizabad,Onion,Red,FAQ,1100.0,1200.0,1150.0,07 Apr 2018,2018,Apr,7
719955,Ayodhya,Faizabad,Onion,Red,FAQ,1000.0,1100.0,1050.0,08 Apr 2018,2018,Apr,8
717484,Ayodhya,Faizabad,Onion,Red,FAQ,950.0,1100.0,1000.0,10 Apr 2018,2018,Apr,10
735489,Ayodhya,Faizabad,Onion,Red,FAQ,900.0,1100.0,1000.0,11 Apr 2018,2018,Apr,11
731866,Ayodhya,Rudauli,Onion,Red,FAQ,90.0,100.0,95.0,11 Apr 2018,2018,Apr,11


#### We can see `Apr 14` and `Apr 28` have wrong entry values like 95 instead of 950 and 100 instead of 1000

In [49]:
s = up_5yrs[(up_5yrs['modal_rs_quintal'] < 640) & (up_5yrs['commodity'] == 'Onion')]['modal_rs_quintal'].value_counts(ascending=True)
print(s.sort_index().head(20))

modal_rs_quintal
88.0      1
95.0      1
100.0     1
120.0     3
190.0     1
200.0     1
270.0     1
320.0     1
340.0     5
345.0     1
350.0     3
360.0     3
385.0     1
390.0     1
395.0    30
400.0    66
410.0     1
420.0    11
425.0     1
430.0    10
Name: count, dtype: int64


In [15]:
# Checking how many records are more head(20)than the 99.5 percentile
up_5yrs[(up_5yrs['modal_rs_quintal'] > 5100) & (up_5yrs['commodity'] == 'Onion')]['year'].value_counts()

year
2020    1136
2019     456
2023      58
2021      10
2018       3
2022       3
2024       2
Name: count, dtype: int64

In [17]:
up_5yrs[(up_5yrs['modal_rs_quintal'] > 6000) & (up_5yrs['commodity'] == 'Onion') & (up_5yrs['variety'] == "Red") ].describe()

,min_rs_quintal,max_rs_quintal,modal_rs_quintal,year,day_of_month
count,486.000000,486.000000,486.000000,486.000000,486.000000
mean,6454.761317,6661.491770,6592.065844,2019.841564,9.185185
std,627.556706,629.174215,457.223458,0.855305,9.170474
min,0.000000,0.000000,6010.000000,2018.000000,1.000000
25%,6150.000000,6371.250000,6280.000000,2019.000000,2.250000
50%,6450.000000,6650.000000,6512.500000,2020.000000,5.000000
75%,6720.000000,6950.000000,6840.000000,2020.000000,14.000000
max,10500.000000,10590.000000,10540.000000,2024.000000,31.000000


In [29]:
up_5yrs[(up_5yrs['commodity'] == "Onion") & (up_5yrs['modal_rs_quintal'] > 5100)  ]['variety'].value_counts()

variety
Red         1474
Onion         64
Nasik         54
Other         43
Medium        18
Pusa-Red       9
Small          5
Local          1
Name: count, dtype: int64

### For Onion, we can clearly see the spike in 2020 is mainly due to the price of the Red variety only... 
### Capping the price of the Onion to the 99.5 percentile as 2020 was an abnormal year (COVID) 

## Analysis of Wheat prices

In [30]:
price_distribution = get_price_distribution_for_commodity("modal_rs_quintal", up_5yrs, "Wheat")
df_price_distribution = pd.DataFrame([price_distribution])
df_price_distribution

,commodity,min_value,1st_percentile,5th_percentile,25th_percentile,50th_percentile,75th_percentile,90th_percentile,95th_percentile,99th_percentile,99.5th_percentile,max_value
0,Wheat,17.0,1525.0,1620.0,1800.0,1935.0,2220.0,2415.0,2500.0,2700.0,2760.0,7040.0


In [16]:
up_5yrs[(up_5yrs['modal_rs_quintal'] > 4000) & (up_5yrs['commodity'] == 'Wheat') & (up_5yrs.year == 2023) ]

,district_name,market_name,commodity,variety,grade,min_rs_quintal,max_rs_quintal,modal_rs_quintal,date,year,month,day_of_month
544466,Balrampur,Balrampur,Wheat,Dara,FAQ,5440.0,5680.0,5575.0,10 Nov 2023,2023,Nov,10
561653,Balrampur,Tulsipur,Wheat,Dara,FAQ,7010.0,7050.0,7030.0,15 Nov 2023,2023,Nov,15
567662,Balrampur,Tulsipur,Wheat,Dara,FAQ,7020.0,7050.0,7040.0,08 Nov 2023,2023,Nov,8
599847,Gorakhpur,Gorakhpur,Wheat,Dara,FAQ,5300.0,5425.0,5350.0,19 Jul 2023,2023,Jul,19
602619,Jhansi,Moth,Wheat,Dara,FAQ,3950.0,4200.0,4100.0,18 Mar 2023,2023,Mar,18
608588,Barabanki,Safdarganj,Wheat,Dara,FAQ,6800.0,6900.0,6850.0,18 Mar 2023,2023,Mar,18
614633,Prayagraj,Sirsa,Wheat,Dara,FAQ,5180.0,5250.0,5220.0,06 Mar 2023,2023,Mar,6


In [63]:
up_5yrs[(up_5yrs['commodity'] == 'Wheat') & (up_5yrs.year == 2023) & (up_5yrs.month == 'Mar') & (up_5yrs.market_name == "Moth")].sort_values(by="date")

,district_name,market_name,commodity,variety,grade,min_rs_quintal,max_rs_quintal,modal_rs_quintal,date,year,month,day_of_month
624336,Jhansi,Moth,Wheat,Dara,FAQ,2200.0,2275.0,2250.0,03 Mar 2023,2023,Mar,3
593953,Jhansi,Moth,Wheat,Dara,FAQ,2240.0,2300.0,2270.0,04 Mar 2023,2023,Mar,4
588811,Jhansi,Moth,Wheat,Dara,FAQ,2250.0,2300.0,2275.0,05 Mar 2023,2023,Mar,5
593952,Jhansi,Moth,Wheat,Dara,FAQ,2220.0,2300.0,2270.0,11 Mar 2023,2023,Mar,11
618997,Jhansi,Moth,Wheat,Dara,FAQ,2250.0,2300.0,2275.0,12 Mar 2023,2023,Mar,12
624337,Jhansi,Moth,Wheat,Dara,FAQ,2230.0,2265.0,2250.0,13 Mar 2023,2023,Mar,13
618996,Jhansi,Moth,Wheat,Dara,FAQ,2200.0,2220.0,2200.0,14 Mar 2023,2023,Mar,14
610276,Jhansi,Moth,Wheat,Dara,FAQ,2200.0,2250.0,2220.0,17 Mar 2023,2023,Mar,17
602619,Jhansi,Moth,Wheat,Dara,FAQ,3950.0,4200.0,4100.0,18 Mar 2023,2023,Mar,18
618827,Jhansi,Moth,Wheat,Dara,FAQ,2200.0,2250.0,2230.0,21 Mar 2023,2023,Mar,21


#### From the above, we could see that only on 18th the price was Rs 4100, for the rest of the month, it has been around Rs 2250

In [65]:
up_5yrs[(up_5yrs['commodity'] == 'Wheat') & (up_5yrs.year == 2023) & (up_5yrs.month == 'Nov') & (up_5yrs.market_name == "Tulsipur")].sort_values(by="date")

,district_name,market_name,commodity,variety,grade,min_rs_quintal,max_rs_quintal,modal_rs_quintal,date,year,month,day_of_month
553400,Balrampur,Tulsipur,Wheat,Dara,FAQ,2300.0,2340.0,2320.0,01 Nov 2023,2023,Nov,1
579144,Balrampur,Tulsipur,Wheat,Dara,FAQ,2310.0,2360.0,2340.0,02 Nov 2023,2023,Nov,2
582777,Balrampur,Tulsipur,Wheat,Dara,FAQ,2300.0,2350.0,2330.0,03 Nov 2023,2023,Nov,3
577085,Balrampur,Tulsipur,Wheat,Dara,FAQ,2320.0,2360.0,2340.0,04 Nov 2023,2023,Nov,4
580087,Balrampur,Tulsipur,Wheat,Dara,FAQ,2300.0,2350.0,2330.0,05 Nov 2023,2023,Nov,5
568380,Balrampur,Tulsipur,Wheat,Dara,FAQ,2310.0,2340.0,2320.0,07 Nov 2023,2023,Nov,7
567662,Balrampur,Tulsipur,Wheat,Dara,FAQ,7020.0,7050.0,7040.0,08 Nov 2023,2023,Nov,8
567389,Balrampur,Tulsipur,Wheat,Dara,FAQ,2400.0,2500.0,2450.0,09 Nov 2023,2023,Nov,9
577227,Balrampur,Tulsipur,Wheat,Dara,FAQ,2420.0,2475.0,2450.0,10 Nov 2023,2023,Nov,10
576865,Balrampur,Tulsipur,Wheat,Dara,FAQ,2430.0,2480.0,2460.0,11 Nov 2023,2023,Nov,11


In [52]:
up_5yrs[ (up_5yrs['commodity'] == 'Wheat') & (up_5yrs.variety == 'Dara') ]['modal_rs_quintal'].sort_values(ascending=False)

567662    7040.0
561653    7030.0
608588    6850.0
544466    5575.0
599847    5350.0
           ...  
412257     320.0
610159     230.0
591469     230.0
657281     188.0
465820      17.0
Name: modal_rs_quintal, Length: 278289, dtype: float64

### For Wheat, we can clearly see that it spiked in march July and November of 2023 ,November spike was only in Balrampur 

In [18]:
up_5yrs[(up_5yrs['modal_rs_quintal'] > 4000) & (up_5yrs['commodity'] == 'Potato') ]

,district_name,market_name,commodity,variety,grade,min_rs_quintal,max_rs_quintal,modal_rs_quintal,date,year,month,day_of_month
26760,Mahoba,Mahoba,Potato,Local,FAQ,4720.0,4780.0,4750.0,26 Oct 2019,2019,Oct,26
118733,Sambhal,Chandausi,Potato,Desi,FAQ,3975.0,4075.0,4025.0,28 Mar 2019,2019,Mar,28
124220,Kanpur Dehat,Pukharayan,Potato,Desi,FAQ,4060.0,4260.0,4160.0,03 Nov 2018,2018,Nov,3
126697,Bijnor,Najibabad,Potato,Desi,FAQ,4450.0,4550.0,4500.0,14 May 2019,2019,May,14
128039,Saharanpur,Saharanpur,Potato,Desi,FAQ,4130.0,4350.0,4250.0,20 Oct 2018,2018,Oct,20
138231,Jalaun (Orai),Orai,Potato,Desi,FAQ,4500.0,4600.0,4550.0,16 Nov 2018,2018,Nov,16
147664,Kanpur,Kanpur(Grain),Potato,Desi,FAQ,500.0,4825.0,4750.0,09 Mar 2019,2019,Mar,9
164523,Ghaziabad,Muradnagar,Potato,Desi,FAQ,950.0,9150.0,9050.0,25 Jan 2022,2022,Jan,25
192996,Raebarelli,Lalganj,Potato,Desi,FAQ,4000.0,4100.0,4050.0,20 Nov 2021,2021,Nov,20
240432,Khiri (Lakhimpur),Mohammdi,Potato,Haldwani,FAQ,4000.0,4100.0,4050.0,13 Jul 2024,2024,Jul,13


### We can see there is some abnormal single spike in Jul 2023 but the other one in Jan 2022 with modal price of 9050 is clearly entry error it should have been 905


### From the above data, it is clear that the modal_price > 5100 was pre-dominantly during Covid Period. So we would be capping it to 5100.

In [30]:
up_5yrs[(up_5yrs['commodity'] == 'Potato')&(up_5yrs['modal_rs_quintal'] <200)].sort_values(by="day_of_month")

,district_name,market_name,commodity,variety,grade,min_rs_quintal,max_rs_quintal,modal_rs_quintal,date,year,month,day_of_month
70557,Shahjahanpur,Badda,Potato,Potato,FAQ,152.0,165.0,160.0,01 Feb 2018,2018,Feb,1
64109,Etah,Awagarh,Potato,Desi,FAQ,0.0,0.0,150.0,03 Jan 2018,2018,Jan,3
55743,Etah,Awagarh,Potato,Desi,FAQ,0.0,0.0,150.0,04 Jan 2018,2018,Jan,4
114158,Raebarelli,Lalganj,Potato,Desi,FAQ,1.0,11.0,11.0,12 Sep 2018,2018,Sep,12
139539,Shahjahanpur,Puwaha,Potato,Potato,FAQ,130.0,140.0,135.0,17 Sep 2018,2018,Sep,17
55259,Bulandshahar,Anoop Shahar,Potato,F.A.Q.,FAQ,0.0,0.0,190.0,18 Jan 2018,2018,Jan,18
56863,Bulandshahar,Anoop Shahar,Potato,F.A.Q.,FAQ,0.0,0.0,190.0,19 Jan 2018,2018,Jan,19
105661,Sant Kabir Nagar,Khalilabad,Potato,Potato,FAQ,12.0,1350.0,12.0,24 Sep 2018,2018,Sep,24
55589,Etah,Awagarh,Potato,Potato,FAQ,0.0,0.0,150.0,30 Jan 2018,2018,Jan,30


In [35]:
up_5yrs[(up_5yrs['commodity'] == 'Potato')&(up_5yrs['year'] ==2018) & (up_5yrs['month'] == "Feb") & (up_5yrs['market_name'] == "Badda")].sort_values(by="day_of_month")

,district_name,market_name,commodity,variety,grade,min_rs_quintal,max_rs_quintal,modal_rs_quintal,date,year,month,day_of_month
70557,Shahjahanpur,Badda,Potato,Potato,FAQ,152.0,165.0,160.0,01 Feb 2018,2018,Feb,1
67807,Shahjahanpur,Badda,Potato,Potato,FAQ,450.0,550.0,500.0,02 Feb 2018,2018,Feb,2
55970,Shahjahanpur,Badda,Potato,Potato,FAQ,450.0,550.0,500.0,04 Feb 2018,2018,Feb,4
56620,Shahjahanpur,Badda,Potato,Potato,FAQ,450.0,850.0,500.0,06 Feb 2018,2018,Feb,6
70558,Shahjahanpur,Badda,Potato,Potato,FAQ,350.0,550.0,470.0,07 Feb 2018,2018,Feb,7
88629,Shahjahanpur,Badda,Potato,Potato,FAQ,350.0,450.0,400.0,08 Feb 2018,2018,Feb,8
68152,Shahjahanpur,Badda,Potato,Potato,FAQ,500.0,600.0,520.0,09 Feb 2018,2018,Feb,9
74977,Shahjahanpur,Badda,Potato,Potato,FAQ,800.0,900.0,850.0,11 Feb 2018,2018,Feb,11
57419,Shahjahanpur,Badda,Potato,Potato,FAQ,500.0,580.0,550.0,12 Feb 2018,2018,Feb,12
55972,Shahjahanpur,Badda,Potato,Potato,FAQ,500.0,580.0,550.0,13 Feb 2018,2018,Feb,13


In [27]:
basmati_records = up_5yrs[up_5yrs['variety'] == 'Fine(Basmati)']
print("basmati records: ", len(basmati_records))
print("total records  : ", len(up_5yrs))
print("% of basmati records: ", 100 * len(basmati_records) / len(up_5yrs))

basmati records:  846
total records  :  1226694
% of basmati records:  0.06896585456519719


In [47]:
# up_5yrs[(up_5yrs['modal_rs_quintal'] > 4750) & (up_5yrs['commodity'] == 'Rice')]['year'].value_counts()
# up_5yrs[(up_5yrs['modal_rs_quintal'] > 5000) & (up_5yrs['commodity'] == 'Rice')]['district_name'].value_counts()
up_5yrs[(up_5yrs['variety'] == 'Fine(Basmati)') & (up_5yrs['commodity'] == 'Rice')]['district_name'].value_counts()

district_name
Gautam Budh Nagar    791
Fatehpur              44
Hathras               10
Ayodhya                1
Name: count, dtype: int64

### We clearly see from the data above that 
 - #### There are only 846 rows for `Fine(Basmati)` which is less than 0.1% of rows
 - #### `Fine(Basmati)` vareity of rice mainly sold in `Gautam Budh Nagar` district 
 - #### Price of `Fine(Basmati)` is more than double the price of the other varieties available

### To do proper prediction we will initially not include the `Fine(Basmati)` in the model as it may skew the learnings of the model

In [20]:
up_5yrs[(up_5yrs['commodity'] == "Rice") & (up_5yrs['max_rs_quintal'] > 4798)]['year'].value_counts()

year
2021    266
2020    238
2022    231
2023    182
2024    145
2019     45
2018     15
Name: count, dtype: int64

In [22]:
up_5yrs[(up_5yrs['commodity'] == "Potato") & (up_5yrs['max_rs_quintal'] > 3200)]['year'].value_counts()

year
2020    1470
2024      49
2019      15
2021       9
2018       8
2022       2
2023       2
Name: count, dtype: int64

In [23]:
up_5yrs[(up_5yrs['commodity'] == "Wheat") & (up_5yrs['max_rs_quintal'] > 2810)]['year'].value_counts()

year
2023    1299
2022      82
2024      36
2020      19
2018      11
2019      10
2021       8
Name: count, dtype: int64